In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import glob
import cv2

#from camera_calibration import read_calibration, undistort_image
#from color_and_gradient import pipeline

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip

import ipywidgets as widgets
from IPython.display import HTML, display, clear_output

In [2]:
def plt_imlist(images, fig_size=(30,15)):
    "Plot images horizontally"
    plt.figure()
    plt.rcParams["figure.figsize"] = fig_size
    fig, axs = plt.subplots(1, len(images))
    for i in range(len(images)):
        if len(images[i].shape) == 2:
            # Binary image
            axs[i].imshow(np.dstack((images[i], images[i], images[i]))*255)
        else:
            axs[i].imshow(images[i])
    
    plt.show()   

In [12]:
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    # Convert to HLS color space and separate the S channel
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    
    # Apply Sobel filter in horizontal direction
    sobelx = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient magnitude
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold Saturation channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Stack each channel
    ret = np.zeros_like(s_channel)
    ret[(s_binary == 1) | (sxbinary == 1)] = 1
    
    return ret

# Test on images

In [13]:
images = glob.glob("test_images/**.jpg")

def tuning_on_image(img_idx, sat_thresh, sobel_thresh):
    img = mpimg.imread(images[img_idx])
    processed_img = pipeline(img, s_thresh=(sat_thresh[0], sat_thresh[1]), sx_thresh=(sobel_thresh[0], sobel_thresh[1]))
    plt_imlist([img, processed_img]) 

In [14]:
widgets.interactive(tuning_on_image,
                    img_idx=widgets.IntSlider(min=1, max=len(images)-1, step=1, value=1),  
                    sat_thresh=widgets.IntRangeSlider(min=0, max=255, step=1,value=[170, 255]),
                    sobel_thresh=widgets.IntRangeSlider(min=0, max=255, step=1,value=[20, 100]))

interactive(children=(IntSlider(value=1, description='img_idx', max=7, min=1), IntRangeSlider(value=(170, 255)…

# Test on challenge video

In [15]:
video = VideoFileClip('test_videos/harder_challenge_video.mp4')

def tuning_on_video(frame_idx, sat_thresh, sobel_thresh):
    img = video.get_frame(frame_idx)
    processed_img = pipeline(img, s_thresh=(sat_thresh[0], sat_thresh[1]), sx_thresh=(sobel_thresh[0], sobel_thresh[1]))
    plt_imlist([img, processed_img]) 

In [16]:
widgets.interactive(tuning_on_video,
                    frame_idx=widgets.IntSlider(min=1, max=video.reader.nframes-1, step=1, value=1),  
                    sat_thresh=widgets.IntRangeSlider(min=0, max=255, step=1,value=[170, 255]),
                    sobel_thresh=widgets.IntRangeSlider(min=0, max=255, step=1,value=[20, 100]))

interactive(children=(IntSlider(value=1, description='frame_idx', max=1198, min=1), IntRangeSlider(value=(170,…